# PyMentat を用いた有限要素モデルの作成

In [122]:
# PyMentatモジュールのインポート
# from py_mentat import *
import math

In [123]:
# パラメータの設定
z = [0, 5000, 10000, 15000] # 基準点(mm)
t = [270, 260, 250, 240] # 板厚(mm)
r = [1900, 1850, 1800, 1750] # 半径(mm)
n = [3, 12, 10] # (r, theta, z)方向の区間ごと分割数

In [124]:
# ループのため利用するデータを区間ごとにまとめておく
z1 = z[0:len(z)-1]
z2 = z[1:len(z)]
t1 = t[0:len(t)-1]
t2 = t[1:len(t)]
r1 = r[0:len(r)-1]
r2 = r[1:len(r)]

data = []
for i in range(0, len(z1)):
    data.append([z1[i], z2[i], t1[i], t2[i], r1[i], r2[i]])

In [125]:
# 座標値の作成

nodes = []
# 区間ごとにループを実行(最後の区間は別に)
for (z1, z2, t1, t2, r1, r2) in data[:len(data)-1]:
    # z座標を分割
    z_list = [(z2 - z1) / n[2] * i + z1 for i in range(n[2])]
    
    # z座標におけるr, tを計算
    t_list = [(t2 - t1) / n[2] * i + t1 for i in range(n[2])]
    r_list = [(r2 - r1) / n[2] * i + r1 for i in range(n[2])]
    
    for k in range(len(z_list)):
        # z
        zz = z_list[k]
        
        # r方向の分割値
        rr = r_list[k]
        tt = t_list[k]
        dr_list = [(rr - tt) + tt / n[0] * i for i in range(n[0]+1)]
        
        # theta方向の分割値(radian)
        rad_list = [2 * math.pi / n[1] * i for i in range(n[1])]
        
        for dr in dr_list:
            for rad in rad_list:
                x = dr * math.cos(rad)
                y = dr * math.sin(rad)
                nodes.append([x, y, zz])
                print('%e %e %e ' % (x, y, zz))
        
# 最後の区間
(z1, z2, t1, t2, r1, r2) = data[-1]

# z座標を分割
z_list = [(z2 - z1) / n[2] * i + z1 for i in range(n[2]+1)]
    
# z座標におけるr, tを計算
t_list = [(t2 - t1) / n[2] * i + t1 for i in range(n[2]+1)]
r_list = [(r2 - r1) / n[2] * i + r1 for i in range(n[2]+1)]
    
for k in range(len(z_list)):
    # z
    zz = z_list[k]
        
    # r方向の分割値
    rr = r_list[k]
    tt = t_list[k]
    dr_list = [(rr - tt) + tt / n[0] * i for i in range(n[0]+1)]
        
    # theta方向の分割値(radian)
    rad_list = [2 * math.pi / n[1] * i for i in range(n[1])]
        
    for dr in dr_list:
        for rad in rad_list:
            x = dr * math.cos(rad)
            y = dr * math.sin(rad)
            nodes.append([x, y, zz])
            print('%e %e %e ' % (x, y, zz))


1.400000e+03 0.000000e+00 0.000000e+00 
8.572528e-14 1.400000e+03 0.000000e+00 
-1.400000e+03 1.714506e-13 0.000000e+00 
-2.571758e-13 -1.400000e+03 0.000000e+00 
1.650000e+03 0.000000e+00 0.000000e+00 
1.010334e-13 1.650000e+03 0.000000e+00 
-1.650000e+03 2.020667e-13 0.000000e+00 
-3.031001e-13 -1.650000e+03 0.000000e+00 
1.900000e+03 0.000000e+00 0.000000e+00 
1.163414e-13 1.900000e+03 0.000000e+00 
-1.900000e+03 2.326829e-13 0.000000e+00 
-3.490243e-13 -1.900000e+03 0.000000e+00 
1.412500e+03 0.000000e+00 1.250000e+03 
8.649068e-14 1.412500e+03 1.250000e+03 
-1.412500e+03 1.729814e-13 1.250000e+03 
-2.594720e-13 -1.412500e+03 1.250000e+03 
1.650000e+03 0.000000e+00 1.250000e+03 
1.010334e-13 1.650000e+03 1.250000e+03 
-1.650000e+03 2.020667e-13 1.250000e+03 
-3.031001e-13 -1.650000e+03 1.250000e+03 
1.887500e+03 0.000000e+00 1.250000e+03 
1.155760e-13 1.887500e+03 1.250000e+03 
-1.887500e+03 2.311521e-13 1.250000e+03 
-3.467281e-13 -1.887500e+03 1.250000e+03 
1.425000e+03 0.000000e

In [126]:
# 要素の作成
n_blocks = len(z) - 1

num_z = n[2]
num_r = n[0]
num_theta = n[1]
num_layer = (num_r + 1) * num_theta # 各層ごとの節点数

elems = []
for k in range(num_z * n_blocks): # z方向
    for j in range(num_r): # r方向
        for i in range(num_theta): # theta方向
            if i == num_theta - 1: # 1周回ったら補正
                o = (num_layer * k) + num_theta * j + i + 1 # 基準点
                e = [o, o+num_theta, o+1, o-num_theta+1, o+num_layer, o+num_layer+num_theta, o+num_layer+1, o+num_layer-num_theta+1]
            else:
                o = (num_layer * k) + num_theta * j + i + 1 # 基準点
                e = [o, o+num_theta, o+num_theta+1, o+1, o+num_layer, o+num_layer+num_theta, o+num_layer+num_theta+1, o+num_layer+1]
            elems.append(e)
            print(e)

[1, 5, 6, 2, 13, 17, 18, 14]
[2, 6, 7, 3, 14, 18, 19, 15]
[3, 7, 8, 4, 15, 19, 20, 16]
[4, 8, 5, 1, 16, 20, 17, 13]
[5, 9, 10, 6, 17, 21, 22, 18]
[6, 10, 11, 7, 18, 22, 23, 19]
[7, 11, 12, 8, 19, 23, 24, 20]
[8, 12, 9, 5, 20, 24, 21, 17]
[13, 17, 18, 14, 25, 29, 30, 26]
[14, 18, 19, 15, 26, 30, 31, 27]
[15, 19, 20, 16, 27, 31, 32, 28]
[16, 20, 17, 13, 28, 32, 29, 25]
[17, 21, 22, 18, 29, 33, 34, 30]
[18, 22, 23, 19, 30, 34, 35, 31]
[19, 23, 24, 20, 31, 35, 36, 32]
[20, 24, 21, 17, 32, 36, 33, 29]
[25, 29, 30, 26, 37, 41, 42, 38]
[26, 30, 31, 27, 38, 42, 43, 39]
[27, 31, 32, 28, 39, 43, 44, 40]
[28, 32, 29, 25, 40, 44, 41, 37]
[29, 33, 34, 30, 41, 45, 46, 42]
[30, 34, 35, 31, 42, 46, 47, 43]
[31, 35, 36, 32, 43, 47, 48, 44]
[32, 36, 33, 29, 44, 48, 45, 41]
[37, 41, 42, 38, 49, 53, 54, 50]
[38, 42, 43, 39, 50, 54, 55, 51]
[39, 43, 44, 40, 51, 55, 56, 52]
[40, 44, 41, 37, 52, 56, 53, 49]
[41, 45, 46, 42, 53, 57, 58, 54]
[42, 46, 47, 43, 54, 58, 59, 55]
[43, 47, 48, 44, 55, 59, 60, 56]
[44